## Introduction

Construire un pipeline ETL complet pour récupérer les données de l'API Spotify, les tranformer avec python, les stocker dans snowflake et produire des visualisations pour analyser la popularité des artistes et des titres. 

Spotify est une plateforme de streaming musical et de podcasts proposant de nombreux artistes, leurs albums et titres respectifs. Chaque artiste et titre possède une cote de popularité, et cet exercice est centré sur l'analyse des facteurs qui influencent cette mesure.

## Installation et importation des librairies

In [ ]:
# import libraries
import requests
import numpy as np
import pandas as pd
import psycopg2 as ps
import datetime
import base64
import time
import os
import json
from dotenv import load_dotenv
from urllib.parse import quote_plus
from datetime import date
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd


## 1.Authentification à l’API Spotify

In [50]:
client_id="ffa6d8652d964d5098aab9ba0959c691"
client_secret="9bda46abe41548b18f2db456013767ca"

auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)


## 2. Fonction pour récupérer l'ID d’un artiste

In [ ]:
def get_artist_id(artist_name):
    result = sp.search(q=artist_name, type='artist', limit=1)
    items = result['artists']['items']
    if items:
        return items[0]['id'], items[0]['name']
    return None, None

## 3. Récupérer les top tracks d’un artiste

In [ ]:
def get_top_tracks(artist_id, artist_name):
    results = sp.artist_top_tracks(artist_id, country='SN')  # SN = Sénégal
    tracks = []

    for track in results['tracks']:
        tracks.append({
            'artist_name': artist_name,
            'track_name': track['name'],
            'track_id': track['id'],
            'popularity': track['popularity'],
            'album': track['album']['name'],
            'release_date': track['album']['release_date']
        })

    return tracks

## 4. Artistes à interroger

In [53]:
artist_names = [
    'Youssou Ndour',
    'Baba Maal',
    'Orchestra Baobab',
    'Ismaila Lo',
    'Cheikh Lo',
    'Viviane Chidid',
    'Waly Seck'
]

all_tracks = []

for name in artist_names:
    artist_id, artist_clean_name = get_artist_id(name)
    if artist_id:
        print(f'Récupération des morceaux de : {artist_clean_name}')
        artist_tracks = get_top_tracks(artist_id, artist_clean_name)
        all_tracks.extend(artist_tracks)
    else:
        print(f'Artiste non trouvé : {name}')


Récupération des morceaux de : Youssou N'Dour
Récupération des morceaux de : Baba Maal
Récupération des morceaux de : Orchestra Baobab
Récupération des morceaux de : Ismaël Lô
Récupération des morceaux de : Cheikh Lô
Récupération des morceaux de : Viviane Chidid
Récupération des morceaux de : Waly Seck


In [ ]:
df_tracks = pd.DataFrame(all_tracks)
df_tracks.head()

,artist_name,track_name,track_id,popularity,album,release_date
0,Youssou N'Dour,7 Seconds (feat. Neneh Cherry),4sv1iemiMkkZVTUbV16C8L,64,The Guide (Wommat),1994-05-16
1,Youssou N'Dour,Solidarité,2dDDAZPMPvlHy3FyEzX4a3,56,Lamomali,2017-04-07
2,Youssou N'Dour,"7 Seconds - Live at Live 8, Palais de Versaill...",04AZfJPUUJ6dwY2KHnzBRE,44,"Live 8 (Live, July 2005)",2019-05-28
3,Youssou N'Dour,L'ours,65K1cio9RsXMOGsSGMMjrN,41,Ma vie d'artiste Unplugged,2020-10-30
4,Youssou N'Dour,In Your Eyes,0JNrEabbksF2iVAAxyXIFS,31,Fatteliku (Live in Athens 1987),2013


In [71]:
df_tracks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   artist_name      50 non-null     object        
 1   track_name       50 non-null     object        
 2   track_id         50 non-null     object        
 3   popularity       50 non-null     int64         
 4   album            50 non-null     object        
 5   release_date     50 non-null     object        
 6   extraction_date  50 non-null     datetime64[us]
dtypes: datetime64[us](1), int64(1), object(5)
memory usage: 2.9+ KB


In [ ]:
# date d'extraction 
df_tracks['extraction_date'] = datetime.now()
# on onvertit en string 
df_tracks['extraction_date'] = df_tracks['extraction_date'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [73]:
df_tracks.head()

,artist_name,track_name,track_id,popularity,album,release_date,extraction_date
0,Youssou N'Dour,7 Seconds (feat. Neneh Cherry),4sv1iemiMkkZVTUbV16C8L,64,The Guide (Wommat),1994-05-16,2025-04-21 00:39:57
1,Youssou N'Dour,Solidarité,2dDDAZPMPvlHy3FyEzX4a3,56,Lamomali,2017-04-07,2025-04-21 00:39:57
2,Youssou N'Dour,"7 Seconds - Live at Live 8, Palais de Versaill...",04AZfJPUUJ6dwY2KHnzBRE,44,"Live 8 (Live, July 2005)",2019-05-28,2025-04-21 00:39:57
3,Youssou N'Dour,L'ours,65K1cio9RsXMOGsSGMMjrN,41,Ma vie d'artiste Unplugged,2020-10-30,2025-04-21 00:39:57
4,Youssou N'Dour,In Your Eyes,0JNrEabbksF2iVAAxyXIFS,31,Fatteliku (Live in Athens 1987),2013,2025-04-21 00:39:57


## 5.Load data into Snowflake

In [ ]:
import snowflake.connector

conn = snowflake.connector.connect(
    user='PSNDAO',
    password='XXXXXXXXXXXXXX',  
    account='SBHEXGP-SS86939',
    warehouse='COMPUTE_WH',    
    database='GALSEN_PLAYLIST',
    schema='DATA_SPOTIFY',        
    role='ACCOUNTADMIN'
)

cursor = conn.cursor()


In [75]:
# Préparation de la requête SQL
insert_sql = """
    INSERT INTO SPOTIFY_TOP_TRACKS (
        ARTIST_NAME, TRACK_NAME, TRACK_ID, POPULARITY, ALBUM, RELEASE_DATE, EXTRACTION_DATE
    ) VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

In [ ]:
# Chargement ligne par ligne
for _, row in df_tracks.iterrows():
    cursor.execute(insert_sql, (
        row['artist_name'],
        row['track_name'],
        row['track_id'],
        int(row['popularity']),
        row['album'],
        row['release_date'],
        row['extraction_date']  
    ))


conn.commit()
cursor.close()
conn.close()